In [ ]:
def tokenize_function(example):
    # Create a prompt that shows the error and wrong candidate,
    # then asks for the correct candidate.
    prompt = (
        f"Error: {example['error_message']}\n"
        f"Wrong Candidate: {example['wrong_candidate']}\n"
        "Correct Candidate:"
    )
    # You might combine prompt with the true candidate as target during training (depends on your task formulation).
    # Here we assume the true candidate is the expected completion.
    full_text = prompt + " " + example['true_candidate']
    
    # Tokenize the combined text
    return tokenizer(full_text, truncation=True, padding="max_length", max_length=512)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

model_name = "your-pretrained-model"  # e.g., "huggingface/llama-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model in 4-bit mode. Adjust the parameters as needed.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",   # This will put parts of the model on available GPUs/CPU.
    # You may need to provide additional quantization config if required.
)

# Define LoRA configuration
lora_config = LoraConfig(
    r=16,                    # LoRA rank
    lora_alpha=32,           # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Customize based on your model architecture
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


In [ ]:
from datasets import load_dataset

# Load the dataset (adjust file paths as necessary)
dataset = load_dataset("csv", data_files={"train": "train.csv", "validation": "val.csv"})

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

# Start fine-tuning
trainer.train()
